# Mel-Cepstral Distortion (MCD)

Loading the zipped Data:

In [1]:
from google_drive_downloader.google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='128wwBM1xqg3_-b1jYorkqDLq6swMatk8', dest_path='/content/Dataset',unzip=True) #add file id which are the characters in the url of the drive link
#https://drive.google.com/file/d/128wwBM1xqg3_-b1jYorkqDLq6swMatk8/view?usp=sharing - Bengali to odia

Unzipping...Done.


In [2]:
pip install pyworld

     |████████████████████████████████| 212 kB 4.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pyworld: filename=pyworld-0.3.0-cp37-cp37m-linux_x86_64.whl size=609472 sha256=83f7616e4c6e7c640e87740272b784839104a5af16e774ed3a3aae937928a94e
  Stored in directory: /root/.cache/pip/wheels/e7/7c/11/c775fffa0e1e7b05a6604b4323408a77f80fb4ab304d96b5c6
Successfully built pyworld


In [3]:
pip install pysptk

     |████████████████████████████████| 410 kB 5.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pysptk: filename=pysptk-0.1.20-cp37-cp37m-linux_x86_64.whl size=929914 sha256=42df9398b30e52c5e3723b5df28d53e89ceaa61c4fef7e4035edb38022a27c67
  Stored in directory: /root/.cache/pip/wheels/dd/37/3f/7583b00bf73e73650aad5d268ee46b95dbd6dffbd237e1bf58
Successfully built pysptk


import libraries

In [4]:
import os
import math
import glob
import librosa
import pyworld
import pysptk
import numpy as np
import matplotlib.pyplot as plot

func for loading wav files

In [5]:
SAMPLING_RATE = 22050
FRAME_PERIOD = 5.0

def load_wav(wav_file, sr):
    
    wav, _ = librosa.load(wav_file, sr=sr, mono=True)

    return wav

func for the formulation of MCD

In [6]:
def MCD(x, y):
    log_spec_dB_const = 10.0 / math.log(10.0) * math.sqrt(2.0)
    diff = x - y
    
    return log_spec_dB_const * math.sqrt(np.inner(diff, diff))

Loading original and the synthesized speech data

* provide the path for the original and synthesized files  

In [7]:
ORIGINAL_PATH ='/content/in male voice Bengali and Odia/R1.Regional_Dataset_Male_Voice_in_Bengali' #copy the path of the folder that contains the .wav files of the oriinal voice
SYNTHESIZED_PATH = '/content/in male voice Bengali and Odia/converted_Bengali' #copy the path of the folder that contains the .wav files of the generated voice

In [8]:
Org_speech_wav_file_paths = glob.glob(ORIGINAL_PATH+'/*')
Synth_speech_wav_file_paths = glob.glob(SYNTHESIZED_PATH+'/*')

Org_speech_wav_file_paths[:]


['/content/in male voice Bengali and Odia/R1.Regional_Dataset_Male_Voice_in_Bengali/VCC 15.wav',
 '/content/in male voice Bengali and Odia/R1.Regional_Dataset_Male_Voice_in_Bengali/VCC 9.wav',
 '/content/in male voice Bengali and Odia/R1.Regional_Dataset_Male_Voice_in_Bengali/VCC 14.wav',
 '/content/in male voice Bengali and Odia/R1.Regional_Dataset_Male_Voice_in_Bengali/VCC 6.wav',
 '/content/in male voice Bengali and Odia/R1.Regional_Dataset_Male_Voice_in_Bengali/VCC 7.wav',
 '/content/in male voice Bengali and Odia/R1.Regional_Dataset_Male_Voice_in_Bengali/VCC 11.wav',
 '/content/in male voice Bengali and Odia/R1.Regional_Dataset_Male_Voice_in_Bengali/VCC 10.wav',
 '/content/in male voice Bengali and Odia/R1.Regional_Dataset_Male_Voice_in_Bengali/VCC 18.wav',
 '/content/in male voice Bengali and Odia/R1.Regional_Dataset_Male_Voice_in_Bengali/VCC 13.wav',
 '/content/in male voice Bengali and Odia/R1.Regional_Dataset_Male_Voice_in_Bengali/VCC1.wav',
 '/content/in male voice Bengali an

func for extracting MCEP features (using WORLD analyzer)

In [9]:
def MCEP(wavfile, mcep_target_directory, alpha=0.65, fft_size=512, mcep_size=24):
    
    if not os.path.exists(mcep_target_directory):
        os.makedirs(mcep_target_directory)

    loaded_wav_file = load_wav(wavfile, sr=SAMPLING_RATE)

   
    _, spectral_envelop, _ = pyworld.wav2world(loaded_wav_file.astype(np.double), fs=SAMPLING_RATE,
                                   frame_period=FRAME_PERIOD, fft_size=fft_size)

    
    mcep = pysptk.sptk.mcep(spectral_envelop, order=mcep_size, alpha=alpha, maxiter=0,
                           etype=1, eps=1.0E-8, min_det=0.0, itype=3)

    fname = os.path.basename(wavfile).split('.')[0]
    np.save(os.path.join(mcep_target_directory, fname + '.npy'),
            mcep,
            allow_pickle=False)

storing the mcep features as npy files

In [10]:
alpha = 0.65  
fft_size = 512
mcep_size = 24

dir_org_speech_wav = glob.glob(ORIGINAL_PATH+'/*')
dir_org_speech_mcep = ORIGINAL_PATH+'/mceps_trg'
dir_converted_speech_wav = glob.glob(SYNTHESIZED_PATH+'/*')
dir_converted_speech_mcep =SYNTHESIZED_PATH+'/mceps_conv'

for wav in dir_org_speech_wav:
    MCEP(wav, dir_org_speech_mcep, fft_size=fft_size, mcep_size=mcep_size)

for wav in dir_converted_speech_wav:
    MCEP(wav, dir_converted_speech_mcep, fft_size=fft_size, mcep_size=mcep_size)

MCD calculation calculation using DTW ( Dynamic Time Warping )

In [11]:
def mcd_cal(mcep_org_files, mcep_synth_files, MCD):
    min_cost_tot = 0.0
    total_frames = 0
    
    for i in mcep_org_files:
        x=0
        for j in mcep_synth_files:
            
            split_org_file,  split_synth_file = os.path.basename(i).split('_'), os.path.basename(j).split('_')
            org_speaker, org_speaker_id = split_org_file[0], split_org_file[-1]
            synth_speaker, synth_speaker_id = split_synth_file[0], split_synth_file[-1]
            
            x+=1
            if org_speaker==synth_speaker and org_speaker_id==synth_speaker_id:
                
                org_mcep_npy=np.load(i)
            
                frame_no = len(org_mcep_npy)
                synth_mcep_npy = np.load(j)
                
                min_cost, _ = librosa.sequence.dtw(org_mcep_npy[:, 1:].T, synth_mcep_npy[:, 1:].T, 
                                                   metric=MCD)
    
                min_cost_tot += np.mean(min_cost)
                
                total_frames += frame_no
                
                #print(j,"    ",i,"     ",x,"   ",min_cost_tot,"   ",total_frames)
                
    mcd = min_cost_tot/total_frames
    return mcd, total_frames

Code execution for MCD calculation between original and converted speech

In [13]:
org_file = glob.glob(ORIGINAL_PATH+'/mceps_trg/*')
synth_file= glob.glob(SYNTHESIZED_PATH+'/mceps_conv/*')

cost_function = MCD

mcd, frames_used = mcd_cal(org_file, synth_file, cost_function)



print(f' MCD = {mcd} dB and total of frames {frames_used}')

 MCD = 7.075041897682486 dB and total of frames 31512
